In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""Reference model"""

__author__ = "Anna Buch, Heidelberg University"
__email__ = "a.buch@stud.uni-heidelberg.de"

# Reference model to compare with BN results

"""
NOTE: 
- reference mode to predict rcloss and bred 
- use reference model to verify BN performances for HCMC ds (- maybe not needed), HCMC_OBM ds and CanTHo ds
- for rcloss: train with zero-loss cases , aim is to compae applied approach (chance-of-loss + degree-of-loss + BN)
- always basic set of hyperparameters is used (no tuning),
- evaluate with 10 fold cross-valdiation
"""

import sys, os
from datetime import datetime
import logging
from pathlib import Path
import joblib
import numpy as np
import pandas as pd
import re
import itertools

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import make_scorer, mean_absolute_error

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

sys.path.insert(0, "../")
import utils.feature_selection as fs
import utils.training as t
import utils.evaluation as e
import utils.evaluation_metrics as em
import utils.figures as f
import utils.settings as s
import utils.pipelines as p
import utils.preprocessing as pp

p.main()  # create/update model settings
#s.init()
seed = s.seed

pd.set_option('display.max_columns', None)
plt.figure(figsize=(20, 10))

import contextlib
import warnings
warnings.filterwarnings('ignore')


#### Load R packages to process Conditional Random Forest in python
# *NOTE 1: all needed R packages have to be previously loaded in R*
# *NOTE 2: Make sure that caret package version >= 6.0-81, otherwise caret.train() throws an error*
import rpy2
from rpy2.robjects import pandas2ri, Formula
from rpy2.robjects.packages import importr, data

# get basic R packages
utils = importr('utils')
base = importr('base')
dplyr = importr('dplyr')
stats_r = importr("stats")  # rename due to similar python package

# pandas.DataFrames to R dataframes 
pandas2ri.activate()

# print r df in html
import rpy2.ipython.html
rpy2.ipython.html.init_printing()

# get libraries for CRF processing, ctree_controls etc
party = importr('party')        # Random Forest with Conditional Inference Trees (Conditional Random Forest)
permimp = importr('permimp')  # conditional permutation feature importance
caret = importr('caret') # package version needs to be higher than  >=  6.0-90
nestedcv = importr('nestedcv')
tdr = importr("tdr")



targets = ["Target_relative_contentloss_euro", "Target_businessreduction"]
target = targets[0]


# Get logger  # test: init application
main_logger = f"__reference_model_rcloss__"
logger = s.init_logger(main_logger)

## settings for cv
kfolds_and_repeats = 3,1 #10, 5 # 3, 1  # <k-folds, repeats> for nested cv
inner_cv = RepeatedKFold(n_splits=kfolds_and_repeats[0], n_repeats=kfolds_and_repeats[1], random_state=seed)
# outer_cv = RepeatedKFold(n_splits=kfolds_and_repeats[0], n_repeats=kfolds_and_repeats[1], random_state=seed)
outer_cv = RepeatedKFold(n_splits=kfolds_and_repeats[0], n_repeats=1, random_state=seed) # make same as for R nestedcv.train()

aoi = "hcmc"
# aoi = "hcmc_obm"
# aoi = "cantho"

## TODO make base outdir ./reference_model_results/degree_of_loss
##  out_dir = os.path.join(base_dir, "path")

## save models and their evaluation in following folders:
Path(f"../reference_model_results/{aoi}/models_trained/degree_of_loss/nested_cv_models").mkdir(parents=True, exist_ok=True)
Path(f"../reference_model_results/{aoi}/models_trained/degree_of_loss/final_models").mkdir(parents=True, exist_ok=True)
Path(f"../reference_model_results/{aoi}/models_evaluation/degree_of_loss").mkdir(parents=True, exist_ok=True)
Path(f"../reference_model_results/{aoi}/selected_features/degree_of_loss").mkdir(parents=True, exist_ok=True)


## load cnadidate predictors and target
df_candidates = pd.read_excel("../input_survey_data/input_data_contentloss_tueb.xlsx")
# df_candidates = pd.read_excel("../input_survey_data/input_data_businessreduction_tueb.xlsx")


df_candidates.drop("geometry.1", axis=1, inplace=True)

if target == "Target_relative_contentloss_euro":
    df_candidates[target] = df_candidates[target] * 100  # make target range more comparable with Bred, TODO move to data_clenaing.ipynb

with contextlib.suppress(Exception):
    if target == "Target_relative_contentloss_euro":
        df_candidates.drop("hh_monthly_income_euro", axis=1, inplace=True) 
        df_candidates.drop("shp_registered_capital_euro", axis=1, inplace=True) # drop due to high collinearit with income and sale, and highest number of missing values

with contextlib.suppress(Exception):
    if target == "Target_businessreduction":
        df_candidates.drop("hh_monthly_income_euro", axis=1, inplace=True) 
        df_candidates.drop("shp_content_value_euro", axis=1, inplace=True) 

logger.info(df_candidates.shape)

## Evaluation metrics 
score_metrics = {
    "MAE": make_scorer(mean_absolute_error, greater_is_better=False),
    "RMSE": make_scorer(em.root_mean_squared_error, greater_is_better=False),
    "MBE": make_scorer(em.mean_bias_error, greater_is_better=False),
    # "R2": "r2",
    "SMAPE": make_scorer(em.symmetric_mean_absolute_percentage_error, greater_is_better=False)
}



## empty variables to store model outputs
eval_sets = {}
models_trained = {}
final_models_trained = {}
models_coef = {}
predicted_values = {}
df_feature_importances = pd.DataFrame(index=df_candidates.drop(target, axis=1).columns.to_list())
models_scores = {}

## iterate over piplines. Each pipline contains a scaler and regressor (and optionally a bagging method) 
pipelines = ["pipe_ref_model"]  

## Load set of hyperparamters
hyperparams_set = pp.load_config("../utils/hyperparameter_sets.json")


for pipe_name in pipelines:

    TIME0 = datetime.now()

    ## load model pipelines
    pipe = joblib.load(f'./pipelines/{pipe_name}.pkl')
 
    try:
        model_name = re.findall("[a-zA-Z]+", str(pipe.steps[1][1].__class__).split(".")[-1])[0] # get model name for python models  
    except AttributeError:
        model_name = pipe # get R model name
    
    ## load respective hyperparameter space
    param_space = hyperparams_set[f"{model_name}_hyperparameters"]

    ## if bagging fro model training is used , rename hyperparmeters
    if "bag" in pipe_name.split("_"):
        logger.info(f"Testing {model_name} with bagging")
        param_space = { k.replace('model', 'bagging__estimator') : v for (k, v) in param_space.items()}



    logger.info( f"\n############ Applying {model_name} on {target} ############\n ")

    # save original df for later
    df_Xy = df_candidates

    # rm geometry column which only needed for visualization
    df_Xy = df_Xy.drop("geometry", axis=1)

    # get predictor names
    X_names = df_Xy.drop(target, axis=1).columns.to_list()

    # ## remove zero-loss records only for rcloss
    # if target == "Target_relative_contentloss_euro":
    #     logger.info(f"Removing {df_Xy.loc[df_Xy[target]==0.0,:].shape[0]} zero loss records")
    #     df_Xy = df_Xy.loc[df_Xy[target]!=0.0,:]


    ## drop samples where target is nan
    logger.info(f"Removing {df_Xy[target].isna().sum()} records from entire dataset due that these values are nan in target variable")
    df_Xy = df_Xy[ ~df_Xy[target].isna()]

    ## Elastic Net and Random Forest: drop samples where any value is nan
    if (model_name == "RandomForestRegressor") | (model_name == "ElasticNet") | (model_name == "cforest"):
        print("Dropping records with missing values")
        df_Xy.dropna(inplace=True)

    logger.info(
        f"Finally use {df_Xy.shape[0]} records for feature extraction, from those are {(df_Xy[target][df_Xy[target] == 0.0]).count()} cases with zero-loss or zero-reduction",
    )

    X = df_Xy[X_names]
    y = df_Xy[target]

 
    ## run sklearn model
    print("value ranges of features:", df_Xy.describe())

    ## fit model for unbiased model evaluation and for final model used for Feature importance, Partial Dependence etc.
    mf = t.ModelFitting(
        model=pipe, 
        Xy=df_Xy,
        target_name=target,
        param_space=param_space,
        tuning_score=score_metrics["MAE"], # tune by getting reducing MAE
        cv=inner_cv,
        kfolds_and_repeats=kfolds_and_repeats,
        seed=seed,
    )
    models_trained_ncv = mf.model_fit_ncv()

    # save models from nested cv and final model on entire ds
    joblib.dump(models_trained_ncv, f"../reference_model_results/{aoi}/models_trained/degree_of_loss/nested_cv_models/{model_name}_{target}.joblib")
        
    ## evaluate model    
    me = e.ModelEvaluation(
        models_trained_ncv=models_trained_ncv, 
        Xy=df_Xy,
        target_name=target,
        score_metrics=score_metrics,
        cv=outer_cv,
        kfolds=kfolds_and_repeats[0],
        seed=seed,
    )
    model_evaluation_results = me.model_evaluate_ncv()

    
    ## visual check if hyperparameter ranges are good or need to be adapted
    logger.info(f"Performance of best estimators on outer test-sets:") 
    for i in range(len(model_evaluation_results["estimator"])):
        print(f"{model_name}: ", model_evaluation_results["estimator"][i].best_params_)


    ## store models evaluation 
    models_scores[model_name] =  {
        k: model_evaluation_results[k] for k in tuple("test_" + s for s in list(score_metrics.keys()))
    } # get evaluation scores, metric names start with "test_<metricname>"
    
    ## reverse sklearn.cross_validate() outputted regression scores (e.g. MAE, RMSE, SMAPE, R2)
    models_scores[model_name] = me.negate_scores_from_sklearn_cross_valdiate(models_scores[model_name])


    ## Final model

    ## get final model based on best MAE score during outer cv
    best_idx = list(models_scores[model_name]["test_MAE"]).index(min(models_scores[model_name]["test_MAE"]))
    final_model = model_evaluation_results["estimator"][best_idx]
    logger.info(f"Params of best model: {final_model.best_params_}") 
    final_model = final_model.best_estimator_

    logger.info(f"Performance of best model:") 
    for metric in models_scores[model_name].keys():
        print(metric, models_scores[model_name][metric][best_idx])


    ## predict on entire dataset and save final model
    y_pred = final_model.predict(X) ## need to derive regression coefficients 
    final_models_trained[model_name] = final_model 
    joblib.dump(final_model, f"../reference_model_results/{aoi}/models_trained/degree_of_loss/final_models/{model_name}_{target}.joblib")

    ## Feature importance of best model
    importances = me.permutation_feature_importance(final_model, repeats=5)



    # ## Summarize all models and their evaluation

    ## store fitted models and their evaluation results for later 
    eval_sets[model_name] = df_Xy
    models_trained[f"{model_name}"] = models_trained_ncv
    predicted_values[model_name] = me.residuals

    ## store Feature Importances of each model
    logger.info("\nSelect features based on permutation feature importance")
    df_importance = pd.DataFrame(
        {
            f"{model_name}_importances" : importances[0],   # averaged importnace scores across repeats
            f"{model_name}_importances_std" : importances[1]
        },
        index=X_names,
    )
    df_feature_importances = df_feature_importances.merge(
        df_importance[f"{model_name}_importances"],   # only use mean FI, drop std of FI
        left_index=True, right_index=True, how="outer")
    df_feature_importances = df_feature_importances.sort_values(f"{model_name}_importances", ascending=False)  # get most important features to the top
    logger.info(f"5 most important features: {df_feature_importances.iloc[:5].index.to_list()}")


    logger.info(
    f"\nTraining and evaluation of {model_name} took {(datetime.now() - TIME0).total_seconds() / 60} minutes\n"
    )
            


## Plot performance ranges of all evaluated estimators from outer cross-validation 
logger.info("Creating boxplots for range of performane scores from outer folds of nested cross-validation")
f.boxplot_outer_scores_ncv(
    models_scores,
    outfile=f"../reference_model_results/{aoi}/models_evaluation/degree_of_loss/boxplot_scores4ncv_{target}.png")

# store avergaed scores and std for later usage
model_evaluation = pd.DataFrame(models_scores["RandomForestRegressor"]).mean(axis=0)  # get mean of outer cv metrics (negative MAE and neg RMSE, pos. R2, pos MBE, posSMAPE)
model_evaluation_std = pd.DataFrame(models_scores["RandomForestRegressor"]).std(axis=0)   # get respective standard deviations

model_evaluation = pd.concat([ model_evaluation, model_evaluation_std], axis=1)
model_evaluation.columns = ["RandomForestRegressor_score", "RandomForestRegressor_score_std"]

## rename metrics
model_evaluation.index = model_evaluation.index.str.replace("test_", "")

outfile = f"../reference_model_results/{aoi}/models_evaluation/degree_of_loss/performance_{target}.xlsx"
model_evaluation.round(3).to_excel(outfile, index=True)
# logger.info(f"Outer evaluation scores of nested cross-validation (mean) :\n {model_evaluation.round(3)} \n.. saved to {outfile}")
logger.info(f"Outer evaluation scores of nested cross-validation (median) :\n {model_evaluation.round(3)} \n.. saved to {outfile}")



     
## Feature Importances 
### drop features which dont reduce the loss
df_feature_importances_plot = df_feature_importances
print(df_feature_importances.head(3))
df_feature_importances_plot = df_feature_importances_plot.loc[df_feature_importances_plot[f"{model_name}_importances"] > 0.0, : ] 
df_feature_importances_plot = df_feature_importances_plot.sort_values(f"{model_name}_importances", ascending=True)


## TODO update with plt_fi() func as soons a its more flexible in number of models passed to func()
sns.set_style("whitegrid", {'axes.grid' : False})
plt.figure(figsize=(30, 22), facecolor="w")
fig = df_feature_importances_plot.plot.barh(
    color="darkblue",
    width=0.5,
    alpha=.7,
    )
plt.xlabel("Importance")
plt.ylabel("")
# plt.title(f"Feature Importances for {target.replace('_',' ')}")

top_bar = mpatches.Patch(
    color="darkblue", 
    label=f"{model_name.replace('cR','c R')}", alpha=.7,
)
plt.tick_params(axis='x', which='major', labelsize=12)
plt.tick_params(axis='y', which='major', labelsize=12)
plt.legend(handles=[top_bar], loc="lower right")
plt.tight_layout()
plt.grid(False)
plt.show()
 
fig.get_figure().savefig(
    f"../reference_model_results/{aoi}/models_evaluation/degree_of_loss/feature_importances_{target}.png", 
    bbox_inches="tight")
plt.close()

## Save final feature space 
### The final selection of features is used later for the non-parametric Bayesian Network

## drop records with missing target values
logger.info(f"Dropping {df_candidates[f'{target}'].isna().sum()} records from entire dataset due that these values are nan in target variable")
df_candidates = df_candidates[ ~df_candidates[target].isna()]
logger.info(f"Keeping {df_candidates.shape[0]} records and {df_candidates.shape[1]} features")


## sort features by their overall importance (weighted sum across across all features) 
final_feature_names = df_feature_importances.sort_values(ascending=False).index##[:10]

## save importnat features, first column contains target variable
fs.save_selected_features(
    df_candidates.drop(target, axis=1), # TODO adpat function that target is only once added
    pd.DataFrame(df_candidates, columns=[target]), 
    final_feature_names,
    filename=f"../reference_model_results/{aoi}/selected_features/degree_of_loss/final_predictors_{target}.xlsx"
)



### Empirical ~ predicted
## use y_pred cross-valdiated from outer folds, mulitplied by 100 for more readable output
for k,v in predicted_values.items():
    print(f"\n{k} predicted target from cross-valdiated outer folds:")
    print(em.empirical_vs_predicted(predicted_values[k]["y_true"], predicted_values[k]["y_pred"]))


# ### Plot prediction error 
f.plot_residuals(
    residuals=predicted_values, 
    model_names_abbreviation=["RandomForestRegressor"],  
    model_names_plot=["Random Forest"],
    outfile=f"../reference_model_results/{aoi}/models_evaluation/degree_of_loss/residuals_{target}.png"
)



12-10-2023 01:14:13 - __reference_model_rcloss__ - INFO - (312, 15)
12-10-2023 01:14:13 - __reference_model_rcloss__ - INFO - 
############ Applying RandomForestRegressor on Target_relative_contentloss_euro ############
 
12-10-2023 01:14:13 - __reference_model_rcloss__ - INFO - Removing 0 records from entire dataset due that these values are nan in target variable
12-10-2023 01:14:13 - __reference_model_rcloss__ - INFO - Finally use 294 records for feature extraction, from those are 180 cases with zero-loss or zero-reduction


Dropping records with missing values
value ranges of features:        Target_relative_contentloss_euro  inundation_duration_h   
count                        294.000000             294.000000  \
mean                           4.653852               9.764286   
std                           13.409001              26.058117   
min                            0.000000               0.200000   
25%                            0.000000               2.000000   
50%                            0.000000               3.000000   
75%                            1.960361               6.000000   
max                           91.672324             240.000000   

       water_depth_cm  flowvelocity  contaminations  flood_experience   
count      294.000000    294.000000      294.000000        294.000000  \
mean        31.979592      0.311565        0.969388         83.510204   
std         24.436299      0.126392        0.407793         52.945766   
min          1.000000      0.100000        0.00000

12-10-2023 01:14:28 - __reference_model_rcloss__ - INFO - Performance of best estimators on outer test-sets:
12-10-2023 01:14:28 - __reference_model_rcloss__ - INFO - Params of best model: {'model__random_state': 42, 'model__n_estimators': 100, 'model__criterion': 'squared_error'}
12-10-2023 01:14:28 - __reference_model_rcloss__ - INFO - Performance of best model:


RandomForestRegressor:  {'model__random_state': 42, 'model__n_estimators': 100, 'model__criterion': 'squared_error'}
RandomForestRegressor:  {'model__random_state': 42, 'model__n_estimators': 100, 'model__criterion': 'squared_error'}
RandomForestRegressor:  {'model__random_state': 42, 'model__n_estimators': 100, 'model__criterion': 'squared_error'}
test_MAE 7.089206299583531
test_RMSE 14.353581755301047
test_MBE -0.04842822396011695
test_SMAPE 79.35602009863008


12-10-2023 01:14:30 - __reference_model_rcloss__ - INFO - 
Select features based on permutation feature importance
12-10-2023 01:14:30 - __reference_model_rcloss__ - INFO - 5 most important features: ['bage', 'shp_avgmonthly_sale_euro', 'water_depth_cm', 'flowvelocity', 'precautionary_measures_lowcost']
12-10-2023 01:14:30 - __reference_model_rcloss__ - INFO - 
Training and evaluation of RandomForestRegressor took 0.28753259999999997 minutes

12-10-2023 01:14:30 - __reference_model_rcloss__ - INFO - Creating boxplots for range of performane scores from outer folds of nested cross-validation


In [2]:
model_name

'RandomForestRegressor'